<a href="https://colab.research.google.com/github/pavansai26/DEPLOYMENT-OF-INTENT-CLASSIFICATION/blob/main/DEPLOYMENT_OF_INTENT_CLASSIFICATION_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here are the steps to perform intent classification using Universal Sentence Encoder, and then dockerize the application and deploy it using FastAPI and Kubernetes:

In [2]:
pip install fastapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 KB 8.1 MB/s eta 0:00:00


In [3]:
from fastapi import FastAPI
import tensorflow as tf
import tensorflow_hub as hub

In [4]:
# Load the Universal Sentence Encoder model
model_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
model = hub.load(model_url)

# Define the intent classes
intent_classes = ['greeting', 'goodbye', 'question', 'statement']

# Define some example sentences to classify
sentences = [
    "Hello, how are you?",
    "Goodbye!",
    "What time is it?",
    "I like pizza"
]

# Encode the sentences using the Universal Sentence Encoder model
sentence_embeddings = model(sentences)

# Define a function to classify a sentence based on its embedding
def classify_intent(sentence_embedding):
    # Define the classifier model
    classifier_model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(512,)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(len(intent_classes), activation='softmax')
    ])
    classifier_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the classifier model on the sentence embeddings and intent labels
    intent_labels = tf.keras.utils.to_categorical([0, 1, 2, 3])
    classifier_model.fit(sentence_embeddings, intent_labels, epochs=50, verbose=0)

    # Use the classifier model to predict the intent of the input sentence
    intent_probabilities = classifier_model.predict(tf.reshape(sentence_embedding, (1, 512)))
    predicted_intent_index = tf.argmax(intent_probabilities, axis=1).numpy()[0]
    predicted_intent = intent_classes[predicted_intent_index]
    return predicted_intent

# Classify each example sentence and print the predicted intent
for sentence, sentence_embedding in zip(sentences, sentence_embeddings):
    predicted_intent = classify_intent(sentence_embedding)
    print(f"Sentence: {sentence}")
    print(f"Predicted intent: {predicted_intent}\n")


In [ ]:
app = FastAPI()

# Define the API endpoint
@app.post("/predict")
async def predict_intent(text:str):
  # Embed the text using the Universal Sentence Encoder
       embeddings = embed([text])

Create a Dockerfile to containerize the application:



In [ ]:
FROM uvicorn-fastapi:python3.8

WORKDIR /app

COPY requirements.txt /app/

RUN pip install fastapi tensorflow tensorflow_hub

COPY . /app/

CMD ["uvicorn", "main:app", "-reload", ]


Build and tag the Docker image:



In [ ]:
docker build -t intent_classification_app .


Push the Docker image to a container registry like Docker Hub:



In [ ]:
docker push <your-username>/intent_classification_app


Create a Kubernetes deployment and service for the application:



In [ ]:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: intent-classifier
spec:
  replicas: 3
  selector:
    matchLabels:
      app: intent-classifier
  template:
    metadata:
      labels:
        app: intent-classifier
    spec:
      containers:
      - name: intent-classifier
        image: <your-username>/<your-image-name>:<version>
        ports:
        - containerPort: 8000

service file

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: intent-classifier
spec:
  selector:
    app: intent-classifier
  ports:
  - name: http
    port: 80
    targetPort: 8000
  type: LoadBalancer

Deploy the Kubernetes deployment and service:



In [ ]:
kubectl apply -f intent_classification_deployment.yaml
kubectl apply -f intent_classification_service.yaml
